In [1]:
import os
import cv2
import numpy as np
import random
import testGMM
from gaussian import *

# load data
train_dir = "train_images"# path to the train image dataset
test_dir = "test_images"# path to the train image dataset
# output directory
output_dir = "results"


# Randomly initialize gaussian distribution
# returns a 1 x 3 matrix, 
# where the first entry is an int, second entry is  a 1x3 matrix, 
# third entry is a 3x3 matrix
def initialize():
    mean = np.array([random.randint(1, 255), random.randint(1, 255), random.randint(1, 255)])
    # generate a random positive-semidefinete matrix as covariance matrix
    A = np.random.random((3, 3)) * 60
    cov = np.dot(A, A.transpose())
    scaling = (random.random() * 5.0)
    return [scaling, mean, cov]

In [2]:
# Select a random pixel for initialization
# returns a 1 x 3 matrix, 
# where the first entry is an int, second entry is  a 1x3 matrix, 
# third entry is a 3x3 matrix
def initialize_use_pixel(X):
    N, D = X.shape
    mean = X[random.randint(0, N-1)]
    # generate a random positive-semidefinete matrix as covariance matrix
    A = np.random.random((3, 3)) * 60
    cov = np.dot(A, A.transpose())
    scaling = (random.random() * 5.0)
    return [scaling, mean, cov]

In [3]:

# return true if MLE converges. Return false otherwise
def check_convergence(total_mean, prev_total_mean, tau, iter):
    sum = np.sum(np.apply_along_axis(np.linalg.norm,1, total_mean - prev_total_mean))
    # print("Current Mean: \n", total_mean)
    # print("Previous Mean: \n", prev_total_mean)
    print("Iter " + str(iter) + ": Convergence difference= ", sum)
    return sum <= tau

# In order to apply "np.apply_along_aixs" function with argument input, we have to define our own along_axis function
def along_axis(M, argument):
    return np.apply_along_axis(expoent_vectorized, 2, M, argument)

# mean_diff here is transposed, i.e. mean_diff_transposed.shape = (1, 3)
def expoent_vectorized(mean_diff_transposed, sigma_inv):
    _mean_diff = np.asmatrix(mean_diff_transposed).T
    result = np.asscalar((-0.5) * ((_mean_diff.T) @ sigma_inv @ _mean_diff))
    return result

def covariance_vectorized (mean_diff_transposed):
    mean_diff = np.asmatrix(mean_diff_transposed).T
    return [mean_diff@(mean_diff.T)]



In [4]:
# extract orange pixels from training images
# return Nx3 array
def extract_orange_pixels():
    # store orange pixels, each row is a pixel
    orange_pixels = np.array([])

    for img_name in os.listdir(train_dir):
        if "mask" in img_name:
            continue
        img = cv2.imread(os.path.join(train_dir, img_name))
        # load mask for it
        img_mask = cv2.imread(os.path.join(train_dir, img_name.split(".")[0]+"_mask.png"))
        # reshape to num of rows = num of pixels, num of column = 3 (BGR)
        X = img.transpose(2,0,1).reshape(3,-1).T 
        # reshape and sum mask to 1d array
        X_mask = img_mask.transpose(2,0,1).reshape(3,-1).T.sum(1) 
        # if empty array
        if orange_pixels.size == 0:
            orange_pixels = X[X_mask>50] # get pixels that are not black in mask
        else:
            orange_pixels = np.append(orange_pixels, X[X_mask>20], axis =0)

    return orange_pixels

In [5]:
'''
parameters:
K: int, number of guassian distribution
max_iter: int, maximum number of step in optimization
X: Nx3 array of all orange pixels
'''
def trainGMM(K, max_iter, X, tau_train):
    # Structure of params:
    # [[scale,mean,covariance],[scale,mean,covariance], [scale,mean,covariance]...]
    # scale is a int. Mean is a 3x1 ndarray. Covariance is a 3x3 ndarray
    params = [initialize_use_pixel(X) for cluster in range(K)]
    prev_mean = None
    iter = 0
    N, D = X.shape
    
    while iter < max_iter :
        weights = np.array([]) # i-th row is weight for i-th cluster
        # print("--- Starting Iter #",  iter, " ---")
        if iter>0 and check_convergence(np.array([cluster_mean for cluster_scaling, cluster_mean, cluster_cov in params]), prev_mean, tau_train, iter): break
        
        # Expectation step - get cluster weight
        for cluster in range(K):
            cluster_scaling, cluster_mean, cluster_cov = params[cluster]
            # if there is error, then re-initialize
            if np.isnan(cluster_mean).any():
                return trainGMM(K, max_iter, X, tau_train)
            # Calculate likelihood, each pixel is row of X
            try:
                constant_in_likelihood = 1/(math.sqrt(((2*math.pi)**3)* np.linalg.det(cluster_cov))) 
                sigma_inv = np.linalg.inv(cluster_cov)
            except:
                # catch error
                print("division by zero or singular value")
                print(cluster_mean)
                print(cluster_cov)
            X_diff = X-cluster_mean
            exponent = (-0.5)*(np.dot(X_diff, sigma_inv) * X_diff).sum(1) 
            likelihood = constant_in_likelihood * np.exp(exponent)
            # weight for a single cluster
            cluster_weights = cluster_scaling * likelihood
            # append cluster weight to weights
            weights = cluster_weights.reshape(1,-1) if weights.size == 0 else np.append(weights, cluster_weights.reshape(1,-1), axis=0)
        weights = weights/weights.sum(0)
        
        # update prev total mean
        prev_mean = np.array([cluster_mean for cluster_scaling, cluster_mean, cluster_cov in params])
        
        # Maximization step - get new scaling, mean, and cov for each cluster
        for cluster in range(K):
            # sum of all the weights given a cluster
            sum_weights = np.sum(weights[cluster]) 
            # Calculate new mean: 
            new_mean = np.multiply(X, weights[cluster].reshape(N,1)).sum(0) / sum_weights
            # if a cluster's R value is too close to 255, change it to 254.5. A R value of 255 will cause determinant to be 0
            if new_mean[2]>254.5: new_mean[2] = 254.5
            # calculate new cov
            X_diff = X-params[cluster][1]
            new_cov = np.dot(np.multiply(X_diff, weights[cluster].reshape(N,1)).T, X_diff)/ sum_weights
            # calculate new scale
            new_scaling = sum_weights/N
            # update params
            params[cluster] = (new_scaling, new_mean, new_cov)

        iter += 1
        
    print("final means")
    print(np.array([cluster_mean for cluster_scaling, cluster_mean, cluster_cov in params]))
    return params

In [6]:
# test on test images
# params: [[scale,mean,covariance],[scale,mean,covariance], [scale,mean,covariance]...]
# scale is a int. Mean is a 3x1 ndarray. Covariance is a 3x3 ndarray
def testGMM(params, tau_test, K, prior):
    for img_name in os.listdir(test_dir):
        img = cv2.imread(os.path.join(test_dir, img_name))
        l, w, h = img.shape # original shape of 2D image
        X = img.transpose(2,0,1).reshape(3,-1).T # reshape to num of rows = num of pixels, num of column = 3 (RGB)
        N, D = X.shape
        #img = X.reshape(l, w, -1) # reshape back to 2d image
        likelihood = np.zeros(N)
        
        for cluster in range(K):
            cluster_scaling, cluster_mean, cluster_cov = params[cluster]
            # calculate likelihood using gaussian distribution
            # each pixel is row of X
            constant_in_likelihood = 1/(math.sqrt(((2*math.pi)**3)* np.linalg.det(cluster_cov))) 
            sigma_inv = np.linalg.inv(cluster_cov)
            X2 = X-cluster_mean
            exponent = (-0.5)*(np.dot(X2, sigma_inv) * X2).sum(1) 
            cluster_likelihood = cluster_scaling *constant_in_likelihood * np.exp(exponent)
            likelihood += cluster_likelihood
        
        # posterior
        posterior = prior* likelihood 

        # mask (reshape back to 2D image)
        mask = posterior.reshape(l, w) 
        #print(mask)
        
        # apply mask
        img[mask < tau_test] = 0

        ##  show masked img
        image_name = os.path.join(output_dir,"GMM_"+ str(img_name))
        cv2.imshow(image_name, img)
        cv2.imwrite(image_name, img)
        cv2.waitKey(0)
        print("Finish Generating mask for image ", str(img_name))
        
        # TODO: measure depth and plot GMM
        # depth = measureDepth.measureDepth(cluster, test_img)
    
    print("All Images Completed")
    return None, None, None

In [7]:
if __name__ == "__main__":
    # User defined threshold
    tau_train = 0.7
    tau_test = 0.0000004
    prior = 0.5
    K = 20
    max_iter = 500

    # load training data
    X = extract_orange_pixels()
    # train
    params = trainGMM(K, max_iter, X, tau_train)
    print("Finish Training")
    
    # test
    if not (os.path.isdir(output_dir)):
        os.mkdir(output_dir)
    cluster, mask, depth = testGMM(params, tau_test, K, prior)
    
    # TODO: measure depth and plot GMM

Iter 1: Convergence difference=  910.0069868420192
Iter 2: Convergence difference=  365.22017542948765
Iter 3: Convergence difference=  126.06842947953274
Iter 4: Convergence difference=  77.33926226596047
Iter 5: Convergence difference=  53.5428799358009
Iter 6: Convergence difference=  47.26527154948652
Iter 7: Convergence difference=  50.635826550578855
Iter 8: Convergence difference=  31.508728971011397
Iter 9: Convergence difference=  22.299549320829982
Iter 10: Convergence difference=  20.05793337305932
Iter 11: Convergence difference=  18.483794017211466
Iter 12: Convergence difference=  17.16260818432524
Iter 13: Convergence difference=  15.737713502101613
Iter 14: Convergence difference=  14.546211019321419
Iter 15: Convergence difference=  12.325280684779374
Iter 16: Convergence difference=  9.782654606764467
Iter 17: Convergence difference=  9.46035587414659
Iter 18: Convergence difference=  9.377245434155137
Iter 19: Convergence difference=  8.968064077768028
Iter 20: Conve

Iter 161: Convergence difference=  1.615572612525944
Iter 162: Convergence difference=  1.5544682716220801
Iter 163: Convergence difference=  1.497003873797162
Iter 164: Convergence difference=  1.4429632241638575
Iter 165: Convergence difference=  1.3922362641361488
Iter 166: Convergence difference=  1.3447213692208244
Iter 167: Convergence difference=  1.300310830180159
Iter 168: Convergence difference=  1.2588784612824202
Iter 169: Convergence difference=  1.2202655753382554
Iter 170: Convergence difference=  1.184271689885675
Iter 171: Convergence difference=  1.1506585550157136
Iter 172: Convergence difference=  1.1191694912400507
Iter 173: Convergence difference=  1.0895556259312054
Iter 174: Convergence difference=  1.0615972479831983
Iter 175: Convergence difference=  1.0351143494082855
Iter 176: Convergence difference=  1.0099677629247994
Iter 177: Convergence difference=  0.9860554038794782
Iter 178: Convergence difference=  0.9633075250630299
Iter 179: Convergence difference